In [ ]:
%load_ext autoreload

%autoreload 2

In [ ]:
import sys
sys.path.append('../../')

### Use a Slider for Filtering Data Points

In [ ]:
import pandas as pd
import numpy as np
import random

In [ ]:
df = pd.DataFrame({
    'x': np.random.uniform(-1.0, 1.0, size=(100,)).tolist(),
    'y': np.random.uniform(-1.0, 1.0, size=(100,)).tolist(),
    'flag': [random.choice([True, False]) for i in range(100)],
})
df.head()

In [ ]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource

output_notebook()

In [ ]:
from bokeh.models import RangeSlider
from bokeh.layouts import column
from bokeh.events import SelectionGeometry
from hover.utils.bokeh_helper import servable

@servable()
def burner():
    """
    Trying to intersect the last manually specified selection with a slider coords/attribute range.
    """
    slider = RangeSlider(start=-1.0, end=1.0, value=(-0.5, 0.5), step=0.01)
    source = ColumnDataSource(df)
    plot = figure(tools=['poly_select', 'lasso_select', 'pan', 'wheel_zoom'])
    plot.circle(source=source)
    
    last_manual_selection = set()
    
    def subroutine(lower, upper):
        filter_l = set(np.where(df['y'] > lower)[0])
        filter_u = set(np.where(df['y'] < upper)[0])
        filtered = filter_l.intersection(filter_u)
        return filtered
    
    def selection_callback(event):
        """
        CAUTION: this has to overwrite the last manual selection.
        Hence only manual selections should trigger this callback.
        """
        last_manual_selection.clear()
        last_manual_selection.update(source.selected.indices.copy())
        filtered = subroutine(*slider.value)
        print('A')
        source.selected.indices = list(filtered.intersection(last_manual_selection))
        
    def foo(event):
        print('B')
        
    def slider_callback(attr, old, new):
        to_select = subroutine(*new)
        if last_manual_selection:
            to_select = to_select.intersection(last_manual_selection)
        source.selected.indices = list(to_select)
    
    plot.on_event(SelectionGeometry, selection_callback)
    plot.on_event(SelectionGeometry, foo)
    slider.on_change('value', slider_callback)
    
    return column(slider, plot)

In [ ]:
handle = burner()
show(handle)